In [1]:
#To install wandb package ,which is helpful in generating plots and report.
!pip install wandb

In [2]:
# Essentials
import numpy as np
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split

import random
import imageio
import os
import cv2
import glob
# random.seed(42)

In [3]:
# Load Dataset
!git clone https://github.com/ashwanth10/INaturalist_Dataset.git

fatal: destination path 'INaturalist_Dataset' already exists and is not an empty directory.


In [4]:
# Labels for INaturalist Dataset
class_labels = {
    0: 'Amphibia',
    1: 'Animalia',
    2: 'Arachnida',
    3: 'Aves',
    4: 'Fungi',
    5: 'Insecta', 
    6: 'Mammalia', 
    7: 'Mollusca', 
    8: 'Plantae',
    9: 'Reptilia'
}

num_classes = 10
img_size = 128
directory_train = 'INaturalist_Dataset/train'
directory_test  = 'INaturalist_Dataset/val'

# Load training data
x_train = []
y_train = []
for label, name in class_labels.items():
   list_images = os.listdir(directory_train + '/' + name)
   for image_name in list_images:
       try:
          image = imageio.imread(directory_train +'/'+name+'/'+image_name)
       except: 
          continue
       if np.ndim(image) == 3:
          x_train.append(cv2.resize(image, (img_size, img_size)))
          y_train.append(label)
       else: 
          # Some images are black and white
          print(image_name)

# Load testing data
x_test = []
y_test = []
for label, name in class_labels.items():
   list_images = os.listdir(directory_test + '/' + name)
   for image_name in list_images:
       try:
          image = imageio.imread(directory_test +'/'+name+'/'+image_name)
       except: 
          continue
       if np.ndim(image) == 3:
          x_test.append(cv2.resize(image, (img_size, img_size)))
          y_test.append(label)
       else: 
          # Some images are black and white
          print(image_name)

aa66b3ba5ccffdb3c41c1d3acf7dd569.jpg
2fad1ed3a05629c089cd530f297e27e6.jpg
d192b73b0af85824c65e91f8795d8df7.jpg


In [5]:
#converting training list into numpy arrays.
x_train = np.array(x_train)
y_train = np.array(y_train)

#converting test list into numpy arrays.
x_test = np.array(x_test)
y_test = np.array(y_test)


In [6]:
# Set seed
np.random.seed(0)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size=0.1, 
                                                  stratify=y_train,
                                                  random_state=42)

In [8]:
# normalize training data
x_train = x_train / 255
x_val = x_val / 255

#normalize testing data
x_test = x_test / 255

In [9]:
# One-hot encoding for training data
targs = np.zeros((len(y_train), 10, 1))
for i in range(len(y_train)):
    targs[i][y_train[i]][0] = 1 # preparing the target matrix 

y_train = targs

In [10]:
# One-hot encoding for validation data
targs_val = np.zeros((len(y_val), 10, 1))
for i in range(len(y_val)):
    targs_val[i][y_val[i]][0] = 1 # preparing the target matrix 

y_val = targs_val

In [11]:
# One-hot encoding for testing data
targs_test = np.zeros((len(y_test), 10, 1))
for i in range(len(y_test)):
    targs_test[i][y_test[i]][0] = 1 # preparing the target matrix 

y_test = targs_test

In [12]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[(3,3),(3,3),(3,3),(3,3),(3,3)], [(3,3),(5,5),(5,5),(7,7),(7,7)], [(7,7),(7,7),(5,5),(5,5),(3,3)], [(3,3),(5,5),(7,7),(9,9),(11,11)] ]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': ['true','false']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'num_dense':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [13]:
sweep_id = wandb.sweep(sweep_config, entity="cs21m010-cs21m041", project="DL_Assignment_2")

Create sweep with ID: m0gvx7e0
Sweep URL: https://wandb.ai/cs21m010-cs21m041/DL_Assignment_2/sweeps/m0gvx7e0


In [14]:
def addActivation(model, config):
    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

In [15]:
def configureBatchNormalization(model, config):
    if config.batch_norm == 'True':
        model.add(BatchNormalization())

In [21]:
def calculateTestAccuracy(model,x_test,y_test):
  yhat=model.predict(x_test)
  yhat_max = np.argmax(yhat, axis=1)
  y_test_max = np.argmax(y_test, axis=1)
  y_test_max = y_test_max.reshape(-1)
  accuracy= (yhat_max == y_test_max).sum()/ y_test.shape[0]
  print('accuracy: ', accuracy)

In [17]:
def train():

    # Initialize a new wandb run
    wandb.init(config=sweep_config)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    wandb.run.name = 'num_dense_'+ str(config.num_dense)+'_bs_'+str(config.batch_size)+'_ac_'+ config.activation
    
    # Determine input shape
    input_shape = (img_size, img_size , 3)
    
    # clear session
    tensorflow.keras.backend.clear_session()

    # Define the model architecture
    model = Sequential()

    filter = config.filt_org

    for i in range(5):
        model.add(Conv2D(filters = filter[i], kernel_size = config.kernel_size[i],
                         padding = 'same', input_shape = input_shape, 
                         kernel_regularizer=regularizers.l2(config.weight_decay)))
        
        addActivation(model, config)
        configureBatchNormalization(model, config)
        model.add(MaxPooling2D(pool_size=(2, 2)))  

    # FC layer
    model.add(Flatten())
    model.add(Dense(config.num_dense, activation = config.activation, kernel_regularizer = regularizers.l2(config.weight_decay)))
    model.add(Dropout(config.dropout))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation = "softmax"))

    # Define the optimizer
    optimizer = Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999)
    
    model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

    #data augmentation
    if config.data_augment == 'true':
        datagen = ImageDataGenerator(
            rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False  # randomly flip images
        )
    else:
        datagen = ImageDataGenerator(rescale = 1.0)

    datagen.fit(x_train)
    
    model.fit(
        datagen.flow(x_train, y_train, batch_size = config.batch_size),
        epochs = 1,
        verbose = 1,
        validation_data= (x_val, y_val),
        callbacks = [WandbCallback()]
    )

    calculateTestAccuracy(model,x_test,y_test)
    

In [18]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: bxl2px4q with config:
wandb: 	activation: selu
wandb: 	batch_norm: true
wandb: 	batch_size: 32
wandb: 	data_augment: true
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 32, 32, 32, 32]
wandb: 	kernel_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	learning_rate: 0.0001
wandb: 	num_dense: 256
wandb: 	weight_decay: 0.005
wandb: Currently logged in as: cs21m010-cs21m041 (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


282/282 [==============================] - 53s 150ms/step - loss: 4.8635 - accuracy: 0.1817 - val_loss: 4.3699 - val_accuracy: 0.2690 - _timestamp: 1648464767.0000 - _runtime: 66.0000
accuracy:  0.2665



accuracy,▁
epoch,▁
loss,▁
val_accuracy,▁
val_loss,▁
accuracy,0.18175
best_epoch,0
best_val_loss,4.36993
epoch,0
loss,4.86353
val_accuracy,0.269


In [19]:
# print(y_test)
y_test_max = np.argmax(y_test, axis=1)
print(y_test_max.reshape(-1))

[0 0 0 ... 9 9 9]


In [20]:
y_test_max

array([[0],
       [0],
       [0],
       ...,
       [9],
       [9],
       [9]])